In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('grey_water_management.csv')

In [3]:
X = df[['TDS (mg/l)', 'Turbidity (NTU)', 'pH', 'Depth (m)', 'Flow Discharge (L/min)']].values
y = df['Filter Life Span (hours)'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),    
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(16, activation='relu'),   
    tf.keras.layers.Dense(1)  
])

In [6]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
41/41 [==============================] - 1s 13ms/step - loss: 72328.3359 - val_loss: 105097.5312
Epoch 2/100
41/41 [==============================] - 0s 7ms/step - loss: 73825.7500 - val_loss: 78398.2656
Epoch 3/100
41/41 [==============================] - 0s 7ms/step - loss: 75000.6641 - val_loss: 89838.8125
Epoch 4/100
41/41 [==============================] - 0s 8ms/step - loss: 82296.6172 - val_loss: 95839.7812
Epoch 5/100
41/41 [==============================] - 0s 7ms/step - loss: 76903.9844 - val_loss: 74132.9062
Epoch 6/100
41/41 [==============================] - 0s 7ms/step - loss: 68887.5156 - val_loss: 72912.2188
Epoch 7/100
41/41 [==============================] - 0s 7ms/step - loss: 67146.6562 - val_loss: 72846.7500
Epoch 8/100
41/41 [==============================] - 0s 7ms/step - loss: 69803.8984 - val_loss: 90476.7344
Epoch 9/100
41/41 [==============================] - 0s 7ms/step - loss: 71459.7578 - val_loss: 74044.1484
Epoch 10/100
41/41 [===============

In [18]:
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

13/13 [==============================] - 0s 4ms/step - loss: 15403.7588
Test Loss: 15403.7587890625


In [19]:
y_pred = model.predict(X_test)

13/13 [==============================] - 0s 3ms/step


In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('lifespan.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved successfully.")

INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmp4jp3x8xo\assets


INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmp4jp3x8xo\assets


TensorFlow Lite model saved successfully.


In [21]:
interpreter = tf.lite.Interpreter(model_path="lifespan.tflite")
interpreter.allocate_tensors()

In [22]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [23]:
X_test = X_test.astype(np.float32)

In [24]:
y_pred_tflite = []
for i in range(len(X_test)):
    interpreter.set_tensor(input_details[0]['index'], [X_test[i]])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    y_pred_tflite.append(output_data[0])

In [25]:
y_pred_tflite = np.array(y_pred_tflite)

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y_test, y_pred_tflite)
mae = mean_absolute_error(y_test, y_pred_tflite)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 15403.758638611882
Mean Absolute Error: 103.07312111786264
